In [4]:
from google.cloud import bigquery
import pandas as pd
from datetime import date,datetime,timedelta
from google.cloud import bigquery
from google.oauth2 import service_account
from dotenv import dotenv_values

In [5]:
isLoadingAll=False

# we synch at 02:30(Bangkok) it is  7:30 (UTC is still Yesterday)
predict_datetime = date.today() - timedelta(days = 1) 
#predict_datetime = date.today()
print(f"Get prediction as of {predict_datetime}")


env_path=r'D:\PythonDev\MyQuantFinProject\SMart-AI\.env'
config = dotenv_values(dotenv_path=env_path)

Get prediction as of 2023-07-03


In [6]:
projectId='xxxx'
dataset_id='yyyy'
table_id = f"{projectId}.{dataset_id}.new_result_prediction_incident"

credentials = service_account.Credentials.from_service_account_file(r'xxxxx.json')
client = bigquery.Client(credentials= credentials,project=projectId)
# client = bigquery.Client(project=projectId)

FileNotFoundError: [Errno 2] No such file or directory: 'xxxxx.json'

In [28]:

def load_data_bq(sql:str):

 query_result=client.query(sql)
 df_all=query_result.to_dataframe()
 return df_all

if isLoadingAll:
    sql=f""" select id,predict_severity,prediction_datetime from {table_id} """
else:
    sql=f"""
    select id,predict_severity,prediction_datetime from {table_id} 
     where Date(prediction_datetime)='{predict_datetime}' """
    
print(sql)

df=load_data_bq(sql)
if len(df)>0:
    df.columns=['incident_id','severity_label','prediction_at']
else:
    quit()


    select id,predict_severity,prediction_datetime from smart-data-ml.SMartML.new_result_prediction_incident 
     where Date(prediction_datetime)='2023-04-03' 


In [29]:
# load json file
map_severity_to_class={'Cosmetic':0,'Minor': 1, "Major": 2, "Critical": 3}
revert_class_to_severity = {v: k for k, v in map_severity_to_class.items()}

print(f"Map severity Name to LabelCode: {str(revert_class_to_severity)}")

df['severity_name']=df['severity_label'].map(revert_class_to_severity)

update_at=datetime.now()
df['imported_at']=update_at

Map severity Name to LabelCode: {0: 'Cosmatic', 1: 'Minor', 2: 'Major', 3: 'Critical'}


In [30]:
print(df.info())
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   incident_id     3 non-null      Int64         
 1   severity_label  3 non-null      Int64         
 2   prediction_at   3 non-null      datetime64[ns]
 3   severity_name   3 non-null      object        
 4   imported_at     3 non-null      datetime64[ns]
dtypes: Int64(2), datetime64[ns](2), object(1)
memory usage: 254.0+ bytes
None


,incident_id,severity_label,prediction_at,severity_name,imported_at
0,2575,1,2023-04-03 19:00:29.655106,Minor,2023-04-04 19:13:31.863911
1,2576,1,2023-04-03 19:00:29.655106,Minor,2023-04-04 19:13:31.863911
2,2577,1,2023-04-03 19:00:29.655106,Minor,2023-04-04 19:13:31.863911


In [10]:
import psycopg2
import psycopg2.extras as extras

def get_postgres_conn():
 try:
  conn = psycopg2.connect(
        database=config['DATABASES_NAME'], user=config['DATABASES_USER'],
      password=config['DATABASES_PASSWORD'], host=config['DATABASES_HOST']
     )
  return conn

 except Exception as error:
  print(error)      
  raise error

def add_data_values(df, table,conn):

    tuples = [tuple(x) for x in df.to_numpy()]
    # Comma-separated dataframe columns
    cols = ','.join(list(df.columns))
    # SQL quert to execute
    query  = "INSERT INTO %s(%s) VALUES %%s" % (table, cols)
    #print(query)
    #return query,tuples
    cursor = conn.cursor()
    try:
        extras.execute_values(cursor, query, tuples)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        raise error
        return 0
    
    return 1
    cursor.close()
    
result=add_data_values(df,'app_prediction_ml_severity_incident',get_postgres_conn())

if  result==1:
    print(f"{len(df.index)} items have been imported to database successfully.")
    print("importing data succeeded")

144 items have been imported to database successfully.
importing data succeeded
